In [11]:
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold, GroupKFold,train_test_split
from sklearn.metrics import roc_auc_score, f1_score
import json
from tqdm import tqdm
# import gc
# import time
pd.set_option('display.max_columns', None)
import os
import os.path as osp
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'
from IPython import display
import warnings
warnings.filterwarnings("ignore")
%load_ext autoreload 
%autoreload 2
pd.set_option('display.max_columns', None)
from utils import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
df1 = pd.read_csv("Sichuan/train_user.csv")
df2 = pd.read_csv("Sichuan/train_voc.csv")
df3 = pd.read_csv("Sichuan/train_sms.csv")
df4 = pd.read_csv("Sichuan/train_app.csv")
print(df1.shape)
df1.head()
print(df2.shape)
df2.head()
print(df3.shape)
df3.head()
print(df4.shape)
df4.head()

(6106, 13)


,phone_no_m,city_name,county_name,idcard_cnt,arpu_201908,arpu_201909,arpu_201910,arpu_201911,arpu_201912,arpu_202001,arpu_202002,arpu_202003,label
0,672ddbf02a5544d32e4ecc9433b1981bffe23bf912273a...,绵阳,江油分公司,1,46.06,45.00,45.0,45.00,45.00,45.0,45.0,45.0,0
1,5e1272273e041e82cb275ae877710be98cdaf5b0a8f34d...,德阳,旌阳分公司,1,79.00,79.20,79.1,79.30,41.40,34.1,59.4,60.0,0
2,eaab3472ec87b076e69e6e8bb62b14341638fc63661a6c...,成都,金堂分公司,2,18.85,24.45,22.0,21.40,22.15,21.1,61.3,63.0,0
3,0ce1bb415704178bf44e9c9b431a39b083a132c8e6d99f...,成都,高新分公司,2,19.20,29.65,130.4,173.12,19.00,19.0,19.0,23.2,0
4,28b87f35f63f65096a53e3a4c97eaffd4a6c43ffa7e92d...,德阳,旌阳分公司,1,50.00,50.00,50.0,50.00,50.00,50.0,50.0,50.0,0


(5015430, 8)


,phone_no_m,opposite_no_m,calltype_id,start_datetime,call_dur,city_name,county_name,imei_m
0,3b6bef554a4754fa9977b50d99e6777c9fb26e585961a4...,3572f19c52faeea5111ddd79dfc3643171d00e6d8b3fd9...,2,2019-10-17 17:16:25,13,NaN,NaN,d2940d7cde5092d809af2e4bb8f6936a405382bd541542...
1,3b6bef554a4754fa9977b50d99e6777c9fb26e585961a4...,3572f19c52faeea5111ddd79dfc3643171d00e6d8b3fd9...,2,2019-10-17 17:36:20,16,NaN,NaN,d2940d7cde5092d809af2e4bb8f6936a405382bd541542...
2,ecd1b3f5a38576ef9872d1cb7a762f31869153de736e7a...,c386d11d5203136e924e070afcac734cdaea6691472e84...,1,2019-08-28 09:46:50,18,NaN,NaN,071e8dcb4010c643c48339cb8abf4a69e03b37afade98d...
3,41a97bdfe77f3210deba78689baecb110922878be2f14c...,79ef199a532bcb9a89990d0996cee3d40557f06b9e5265...,1,2019-09-18 16:19:20,34,NaN,NaN,5f054945a5037e688abe0ef36d88a0fd351d476e3320c2...
4,41a97bdfe77f3210deba78689baecb110922878be2f14c...,e273542129cf7fa37c28d3f4c95b7b9e8bcb561bbabc50...,2,2019-09-18 16:18:46,23,NaN,NaN,5f054945a5037e688abe0ef36d88a0fd351d476e3320c2...


(6848509, 4)


,phone_no_m,opposite_no_m,calltype_id,request_datetime
0,0251387744988114430181c3e680a3733001a26fe686bd...,df22edbc0e3dd6bc4f2f453e687b743e8442a54834b64f...,2,2019-08-01 12:13:08
1,0251387744988114430181c3e680a3733001a26fe686bd...,df22edbc0e3dd6bc4f2f453e687b743e8442a54834b64f...,2,2019-08-01 12:13:08
2,0251387744988114430181c3e680a3733001a26fe686bd...,df22edbc0e3dd6bc4f2f453e687b743e8442a54834b64f...,2,2019-08-01 12:13:08
3,296cfae1d838070c4dd05a125a85c3d29bbb95f713c2ea...,1205bb229d750a6bcb3f9c33893b5d68c8fc8a6443b0a7...,2,2019-08-01 17:46:44
4,39de6ef3a87b8e660e42496450c54b731f3621ca708944...,972affd4be9e7596420379b7e7910843759cdfbd81315e...,2,2019-08-01 16:46:30


(3283602, 4)


,phone_no_m,busi_name,flow,month_id
0,416cec0f25b93f08bfd9cff44382c5da3a6346beb16a2c...,NaN,0.000000,NaN
1,26fcb7c6f4125ee5445756d4ff5346c29f2aff7d0f2e31...,NaN,0.000000,NaN
2,1a58c82eeefdb00ea6abf2e6010a8e808b27e1802b764d...,NaN,0.021529,2019-12
3,1a58c82eeefdb00ea6abf2e6010a8e808b27e1802b764d...,微信,0.001846,2019-12
4,1a58c82eeefdb00ea6abf2e6010a8e808b27e1802b764d...,高德导航,0.002288,2019-12


In [13]:
# 处理缺失值
for col in df1.columns:
    if np.sum(df1[col].isna()) > 0:
        print(col, np.sum(df1[col].isna()), df1[col].mode().values[0])
        df1[col] = df1[col].fillna(df1[col].mode().values[0])
        print(col, np.sum(df1[col].isna()))
for col in df2.columns:
    if np.sum(df2[col].isna()) > 0:
        print(col, np.sum(df2[col].isna()), df2[col].mode().values[0])
        df2[col] = df2[col].fillna(df2[col].mode().values[0])
        print(col, np.sum(df2[col].isna()))
for col in df3.columns:
    if np.sum(df3[col].isna()) > 0:
        print(col, np.sum(df3[col].isna()), df3[col].mode().values[0])
        df3[col] = df3[col].fillna(df3[col].mode().values[0])
        print(col, np.sum(df3[col].isna()))
for col in df4.columns:
    if np.sum(df4[col].isna()) > 0:
        print(col, np.sum(df4[col].isna()), df4[col].mode().values[0])
        df4[col] = df4[col].fillna(df4[col].mode().values[0])
        print(col, np.sum(df4[col].isna()))


city_name 289 成都
city_name 0
county_name 289 武侯分公司
county_name 0
arpu_201908 1791 1.0
arpu_201908 0
arpu_201909 1641 1.0
arpu_201909 0
arpu_201910 1436 1.0
arpu_201910 0
arpu_201911 1026 1.0
arpu_201911 0
arpu_201912 854 1.0
arpu_201912 0
arpu_202001 957 19.0
arpu_202001 0
arpu_202002 945 19.0
arpu_202002 0
arpu_202003 737 1.0
arpu_202003 0
city_name 559529 成都
city_name 0
county_name 559529 高新南区
county_name 0
busi_name 381835 微信
busi_name 0
month_id 812 2020-01
month_id 0


In [14]:
phone_no_m_values = df1['phone_no_m'].dropna().unique()
phone_no_m_mapping = {value: idx + 1 for idx, value in enumerate(sorted(phone_no_m_values))}

# 2. 应用映射到 df1 的 phone_no_m
df1['phone_no_m'] = df1['phone_no_m'].map(phone_no_m_mapping)
for df in [df2, df3, df4]:
    df['phone_no_m'] = df['phone_no_m'].map(phone_no_m_mapping)

df2 = df2.drop(columns=['opposite_no_m', 'imei_m'])
df3 = df3.drop(columns=['opposite_no_m'])

In [15]:
columns_to_map = ['city_name', 'county_name', 'busi_name']
all_dfs = [df1, df2, df3, df4]

# 创建全局映射
unique_values = set()
for df in all_dfs:
    for col in columns_to_map:
        if col in df.columns:
            unique_values.update(df[col].dropna().unique())

# 构建映射
unique_values = sorted(unique_values)
value_to_int_mapping = {value: idx + 1 for idx, value in enumerate(unique_values)}

# 应用映射
def map_columns(df, columns, mapping):
    """对指定列应用映射"""
    mapped_df = df.copy()
    for col in columns:
        if col in mapped_df.columns:
            mapped_df[col] = mapped_df[col].map(mapping)
    return mapped_df

mapped_dfs = [map_columns(df, columns_to_map, value_to_int_mapping) for df in all_dfs]

# 更新映射后的 DataFrames
df1, df2, df3, df4 = mapped_dfs

# 2. 处理时间特征并转化为数字
df2['start_datetime'] = pd.to_datetime(df2['start_datetime'], errors='coerce', infer_datetime_format=True)
df3['request_datetime'] = pd.to_datetime(df3['request_datetime'], errors='coerce', infer_datetime_format=True)
df4['month_id'] = pd.to_datetime(df4['month_id'].astype(str), format='%Y/%m/%d', errors='coerce')

# 转化为数字（时间戳）
df2['start_datetime'] = df2['start_datetime'].apply(lambda x: x.timestamp() if not pd.isnull(x) else None)
df3['request_datetime'] = df3['request_datetime'].apply(lambda x: x.timestamp() if not pd.isnull(x) else None)
df4['month_id'] = df4['month_id'].apply(lambda x: x.timestamp() if not pd.isnull(x) else None)

# 3. 保存映射表
mapping_df = pd.DataFrame(list(value_to_int_mapping.items()), columns=['Original_Value', 'Mapped_Value'])


In [16]:
df1
df2
df3
df4

,phone_no_m,city_name,county_name,idcard_cnt,arpu_201908,arpu_201909,arpu_201910,arpu_201911,arpu_201912,arpu_202001,arpu_202002,arpu_202003,label
0,2450,4684,4174,1,46.06,45.00,45.00,45.00,45.00,45.00,45.00,45.00,0
1,2226,3748,3969,1,79.00,79.20,79.10,79.30,41.40,34.10,59.40,60.00,0
2,5589,3798,5151,2,18.85,24.45,22.00,21.40,22.15,21.10,61.30,63.00,0
3,318,3798,5335,2,19.20,29.65,130.40,173.12,19.00,19.00,19.00,23.20,0
4,1019,3748,3969,1,50.00,50.00,50.00,50.00,50.00,50.00,50.00,50.00,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6101,4599,3492,5331,1,1.90,2.50,1.80,2.40,1.80,1.60,1.30,1.40,0
6102,205,3361,4540,4,1.00,1.00,1.00,5.24,1453.50,36.06,22.00,22.00,1
6103,6001,3798,4141,4,1.00,1.00,1.00,1.00,1.00,19.00,19.00,83.14,1
6104,4204,4684,2592,3,269.56,243.01,272.60,246.68,259.30,243.29,243.00,257.71,0


,phone_no_m,calltype_id,start_datetime,call_dur,city_name,county_name
0,1430,2,1.571333e+09,13,3798,5338
1,1430,2,1.571334e+09,16,3798,5338
2,5638,1,1.566986e+09,18,3798,5338
3,1563,1,1.568824e+09,34,3798,5338
4,1563,2,1.568824e+09,23,3798,5338
...,...,...,...,...,...,...
5015425,990,1,1.564949e+09,11,3798,5338
5015426,990,1,1.564949e+09,11,3798,5338
5015427,3297,2,1.570471e+09,244,3798,5338
5015428,3297,2,1.570187e+09,61,3798,5338


,phone_no_m,calltype_id,request_datetime
0,70,2,1.564662e+09
1,70,2,1.564662e+09
2,70,2,1.564662e+09
3,1037,2,1.564682e+09
4,1398,2,1.564678e+09
...,...,...,...
6848504,2523,2,1.585406e+09
6848505,2523,2,1.585406e+09
6848506,2523,2,1.585426e+09
6848507,2523,2,1.585393e+09


,phone_no_m,busi_name,flow,month_id
0,1556,3694,0.000000,1.577837e+09
1,970,3694,0.000000,1.577837e+09
2,667,3694,0.021529,1.575158e+09
3,667,3694,0.001846,1.575158e+09
4,667,5334,0.002288,1.575158e+09
...,...,...,...,...
3283597,1084,391,0.015110,1.577837e+09
3283598,1084,391,0.168604,1.583021e+09
3283599,1084,3694,0.000736,1.575158e+09
3283600,1084,3694,0.000145,1.575158e+09


In [17]:
import pandas as pd

# 假设 df1, df2, df3, df4 是已存在的 DataFrame
# 示例：从 df2-df4 中按 'phone_no_m' 分组并计算平均值
df2_avg = df2.groupby('phone_no_m').mean().reset_index()
df3_avg = df3.groupby('phone_no_m').mean().reset_index()
df4_avg = df4.groupby('phone_no_m').mean().reset_index()

# 准备将 DataFrame 放入列表
dfs = [df1, df2_avg, df3_avg, df4_avg]

# 初始化合并 DataFrame，保留 df1 的所有列
merged_df = dfs[0]

# 遍历其余 DataFrame 并逐步合并
for i, df in enumerate(dfs[1:], start=2):
    # 添加前缀避免冲突，但保持 phone_no_m 列一致
    df = df.add_prefix(f'df{i}_')
    df = df.rename(columns={f'df{i}_phone_no_m': 'phone_no_m'})  # 确保 'phone_no_m' 列名一致

    # 合并当前 DataFrame 到 merged_df，按 'phone_no_m' 进行合并
    merged_df = pd.merge(merged_df, df, on='phone_no_m', how='outer')

# 如果合并后有重复列名，可以处理这些列名
# 遍历列名并在重复的列名后添加后缀
new_columns = []
seen_columns = set()
for col in merged_df.columns:
    if col in seen_columns:
        col = f"{col}_duplicate"
    seen_columns.add(col)
    new_columns.append(col)

# 设置新的列名
merged_df.columns = new_columns

# 打印合并后的 DataFrame
print(merged_df)


      phone_no_m  city_name  county_name  idcard_cnt  arpu_201908  \
0           2450       4684         4174           1        46.06   
1           2226       3748         3969           1        79.00   
2           5589       3798         5151           2        18.85   
3            318       3798         5335           2        19.20   
4           1019       3748         3969           1        50.00   
...          ...        ...          ...         ...          ...   
6101        4599       3492         5331           1         1.90   
6102         205       3361         4540           4         1.00   
6103        6001       3798         4141           4         1.00   
6104        4204       4684         2592           3       269.56   
6105        1571       5094         3527           2       108.54   

      arpu_201909  arpu_201910  arpu_201911  arpu_201912  arpu_202001  \
0           45.00        45.00        45.00        45.00        45.00   
1           79.20        

In [18]:
df = merged_df
df = df.replace(np.inf,1e10)
df = df.replace(-np.inf,-1e10)
for f in df.columns:
    if df[f].isna().sum() > 0:
        print('缺失值', f, df[f].isna().sum())
        # 判断列类型并采用合适的填补方法
        if df[f].dtype in [np.float64, np.float32, np.int64, np.int32]:
            # 数值列：用均值填充
            df[f] = df[f].fillna(df[f].mean())
        else:
            # 非数值列（如字符串或分类数据）：用最频繁值填充
            df[f] = df[f].fillna(df[f].mode()[0])

缺失值 df2_calltype_id 81
缺失值 df2_start_datetime 81
缺失值 df2_call_dur 81
缺失值 df2_city_name 81
缺失值 df2_county_name 81
缺失值 df3_calltype_id 3
缺失值 df3_request_datetime 3


In [19]:
np.random.seed(42)

# 计算总行数
n_rows = len(df)

# 生成随机分布
train_val_test_values = np.random.choice([1, 2, 3], size=n_rows, p=[0.6, 0.2, 0.2])

# 添加到 DataFrame
df['split'] = train_val_test_values

In [20]:
import numpy as np
import pandas as pd
import os
import os.path as osp
import json

df.rename(columns={'label': 'is_sa'}, inplace=True)

# 划分数据集
train = df[df['split'] == 1].reset_index(drop=True)
val = df[df['split'] == 2].reset_index(drop=True)
test = df[df['split'] == 3].reset_index(drop=True)

cols = [f for f in train.columns if f not in ['phone_no_m', 'split', 'is_sa']]

# 转换为 NumPy 数组
train_data = train[cols].values
train_y = train['is_sa'].values
val_data = val[cols].values
val_y = val['is_sa'].values
test_data = test[cols].values
test_y = test['is_sa'].values
test_name = test['phone_no_m']

# 保存到文件
save_dir = './data/SiChuan_avg'
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

np.save(osp.join(save_dir, 'N_train.npy'), train_data)
np.save(osp.join(save_dir, 'N_val.npy'), val_data)
np.save(osp.join(save_dir, 'N_test.npy'), test_data)
np.save(osp.join(save_dir, 'y_train.npy'), train_y)
np.save(osp.join(save_dir, 'y_val.npy'), val_y)
np.save(osp.join(save_dir, 'y_test.npy'), test_y)

# 保存测试集的 phone_no_m 和 is_sa
test_output = pd.DataFrame({
    'phone_no_m': test_name,
    'is_sa': test_y
})
test_output.to_csv(osp.join(save_dir, 'test_true.csv'), index=False)

pd.DataFrame(test_name).to_csv(osp.join(save_dir, 'test_msisdn.csv'), index=False)

# 保存特征名称
feature_name = cols
with open(osp.join(save_dir, 'feature_name.txt'), 'w') as f:
    for i in feature_name:
        f.write(i + '\n')

# 保存数据信息
info = {
    "name": "TeleChallenge",
    "n_num_features": train_data.shape[1],
    "n_classes": 2,
    "task_type": "binclass",
    "n_cat_features": 0,
    "train_size": train_data.shape[0],
    "val_size": val_data.shape[0],
    "test_size": test_data.shape[0]
}
with open(osp.join(save_dir, 'info.json'), 'w') as f:
    json.dump(info, f, indent=4)

10

12

11

12

12

...

16

21

14

9

13